# 1. Entorno

In [1]:
#Creamos un Notebook en Python sobre JupyterLab

# 2. Librerías

In [10]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Librería para manipular los servicios de AWS
import boto3

# 3. Inicio de Sesión sobre SageMaker

In [11]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

# 4. Lectura del Modelo

In [15]:
#Definimos el nombre del entrenamiento al que nos conectamos
nombreDeEntrenamiento = "linear-learner-240927-1556-012-966d28e8"

In [16]:
#Definimos el algoritmo que usamos para entrenar el modelo
algoritmo = "linear-learner"

In [17]:
#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDeEntrenamiento)

In [18]:
#Obtenemos la ruta en donde el modelo se encuentra almacenado
rutaDelModelo = descripcionDeEntrenamiento["ModelArtifacts"]["S3ModelArtifacts"]

In [19]:
#Verificamos
print(rutaDelModelo)

s3://sagemaker-us-east-1-767828747956/linear-learner-240927-1556-012-966d28e8/output/model.tar.gz


In [20]:
#Utilitario para leer modelos
from sagemaker.model import Model

In [21]:
#Leemos el modelo
modelo = Model(
    model_data = rutaDelModelo, #Ruta del modelo
    role = rol, #Rol de ejecución
    image_uri = sagemaker.image_uris.retrieve(algoritmo, region), #Descargamos la implementación del algoritmo desde la región donde entrenamos
    sagemaker_session = sesion #Sesión de SageMaker
)

# 5. Registro del modelo

In [22]:
#Definimos el nombre del modelo
#IMPORTANTE: En "XXX" colocar la fecha de hoy, hay un bug que hace que no puedas ver tu modelo si previamente ya lo creaste y borraste
nombreDelModelo = "modelo-numerico-bda-jac"

In [23]:
#Tipos de registros que el modelo acepta como INPUT para hacer predicciones
#Aceptará registros JSON y registros CSV
tiposDeRegistrosInput = ["application/json", "text/csv"]

In [24]:
#Tipos de registros que el modelo entrega como OUTPUT cuando hace predicciones
#Las predicciones las devolverá en un JSON
tiposDeRegistrosOutput = ["application/json"]

In [25]:
#Tipo de instancias donde el modelo se ejecuta
tipoDeInstanciasDeEjecucion = ["ml.m5.large"]

In [26]:
#Registramos el modelo
registroDelModelo = modelo.register(
    model_package_group_name = nombreDelModelo,
    content_types = tiposDeRegistrosInput, #Tipo de registros INPUT del modelo
    response_types = tiposDeRegistrosOutput, #Tipo de registros OUTPUT del modelo
    inference_instances = tipoDeInstanciasDeEjecucion, #Tipo de servidor en donde se colocará el modelo
    transform_instances = tipoDeInstanciasDeEjecucion #Tipo de servidor en donde el modelo realizará cálculos intermedios
)